# Will a tropical storm make landfall?

## The source dataset

We will use the IBTrACS (International Best Track Archive for Climate Stewardship) tropical storm database hosted at NOAA: https://www.ncdc.noaa.gov/ibtracs/index.php?name=wmo-data. We will be using the NetCDF file that contains all storms. Since NetCDF is a self-describing format, info about the variables contained in this dataset can be instrospected. Additional info about the variables is located here: ftp://eclipse.ncdc.noaa.gov/pub/ibtracs/v03r08/wmo/netcdf/README.netcdf.

The class variable (prediction variable) will be determined from the dataset using the **landfall** variable:

```
    short landfall(storm, time) ;
        landfall:long_name = "Minimum distance to land until next report (0=landfall)" ;
        landfall:units = "km" ;
        landfall:_FillValue = -999s ;
```

In [25]:
import os, sys, re, json, arff
from subprocess import check_output
import netCDF4 as NC
import numpy as np
import pandas as pd
from ipyleaflet import Map, GeoJSON
from astropy.time import Time
from IPython.display import display, HTML


# get netcdf dataset
file = "Allstorms.ibtracs_wmo.v03r08.nc"
ds = NC.Dataset(file)

## Extract features

We will extract features the following features from this dataset:

* name (STRING)
* genesis_basin { 0 = NA - North Atlantic, 1 = SA - South Atlantic, 2 = WP - West Pacific, 3 = EP - East Pacific, 4 = SP - South Pacific, 5 = NI - North Indian, 6 = SI - South Indian }
* sub_basin of first observation { 0 = NA - North Atlantic, 1 = SA - South Atlantic, 2 = WP - West Pacific, 3 = EP - East Pacific, 4 = SP - South Pacific, 5 = NI - North Indian, 6 = SI - South Indian, 7 = AS - Arabian Sea, 8 = BB - Bay of Bengal, 9 = EA - Eastern Australia, 10 = WA - Wester Australia, 11 = CP - Central Pacific, 12 = CS - Carribbean Sea, 13 = GM - Gulf of Mexico, 14 = MM - Missing }
* time of first observation (MJD value) (REAL)
* longitude of first observation (REAL)
* latitude of first observation (REAL)
* distance to land of first observation (REAL)
* msw (maximum sustained wind) of first observation (REAL)
* mcp (minimum central pressure) of first observation (REAL)
* storm nature { 0 = TS - Tropical, 1 = SS - Subtropical, 2 = ET - Extratropical, 3 = DS - Disturbance, 4 = MX - Mix of conflicting reports, 5 = NR - Not Reported, 6 = MM - Missing, 7 = - Missing }
* track_type { 0 = main - cylclogenesis to cyclolysis, 1 = merge - cyclogenesis to merger, 2 = split - split to cyclolysis, 3 = other - split to merger }
* made_landfall { TRUE, FALSE }

In [ ]:
# define dict for discrete features
disc_map = {
    "basin": {
         0: "NA",
         1: "SA",
         2: "WP",
         3: "EP",
         4: "SP",
         5: "NI",
         6: "SI",
         7: "AS",
         8: "BB",
         9: "EA",
        10: "WA",
        11: "CP",
        12: "CS",
        13: "GM",
        14: "MM",
    },
    "nature": {
        0: "TS",
        1: "SS",
        2: "ET",
        3: "DS",
        4: "MX",
        5: "NR",
        6: "MM",
        7: "MM2",
    },
    "track_type": {
        0: "main",
        1: "merge",
        2: "split",
        3: "other",
    }
}

# extract features from each hurricane and separate them by class (made_landfall)
data = []
landfall_count = 0

# compile regular expression for matching unnamed storms
unnamed_re = re.compile(r'(UNNAMED|NOT NAMED)')

for i in range(ds.dimensions['storm'].size):
    
    # get number of observations
    obs = ds.variables['numObs'][i]
    if obs <= 2: continue # skip if there are 2 or less observations
    
    # extract filterable features first
    name = np.array_str(NC.chartostring(ds.variables['name'][i,:]))[2:-1]
    genesis_basin = ds.variables['genesis_basin'][i]
    sub_basin = ds.variables['sub_basin'][i,:obs-1]
    nature = ds.variables['nature_wmo'][i:,obs-1]
    
    # skip records that have missing values in features
    if genesis_basin == 14:
        continue
    # skipping this filter; this filters out the east pacific storms
    #if sub_basin[0] == 14:
    #    continue
    if nature[0] in (4, 5, 6, 7):
        continue
        
    # skip records with unnamed storms
    if unnamed_re.search(name): continue
    
    # extract the rest of the features
    time = ds.variables['time_wmo'][i,:obs-1]
    time_iso = Time(time, format='mjd', scale='utc')
    lon = ds.variables['lon_wmo'][i,:obs-1]
    if abs(lon[0] - lon[-1]) > 180.: # handle wrapping issue
        lon[np.where(lon > 0)] -= 360. # handle wrapping issue
    lat = ds.variables['lat_wmo'][i,:obs-1]
    dist2land = ds.variables['dist2land'][i,:obs-1]
    msw = ds.variables['wind_wmo'][i,:obs-1]
    mcp = ds.variables['pres_wmo'][i,:obs-1]
    tt = ds.variables['track_type'][i]
    landfall = (ds.variables['landfall'][i,:obs-1] == 0).any()
    
    # create GeoJSON of storm track
    ls = { 
        "type": "LineString",
        "coordinates": np.dstack((lon, lat))[0].tolist(),
    }
    
    # create feature for leaflet display;
    # stuff features into a message info for on_hover display
    msg = "{} {} {} {} {} {} {} {} {} {} {} {} {}".format(i, name, obs, genesis_basin, sub_basin[0],
                                                          time_iso[0].iso, lon[0], lat[0], dist2land[0],
                                                          msw[0], mcp[0], nature[0], tt, landfall)
    ls_feature = { 
        "type": "Feature",
        "properties": { "msg": msg },
        "geometry": ls,
    }
    
    # create data dict
    data.append({
        "name": name,
        "genesis_basin": disc_map['basin'][genesis_basin],
        "sub_basin": disc_map['basin'][sub_basin[0]],
        "time": time[0],
        "lon": lon[0],
        "lat": lat[0],
        "dist2land": dist2land[0],
        "msw": msw[0],
        "mcp": mcp[0],
        "nature": disc_map['nature'][nature[0]],
        "track_type": disc_map['track_type'][tt],
        "landfall": landfall,
        "feature": ls_feature,
    })
    
    # tally landfall
    if landfall: landfall_count += 1
    
# print tally
print("total storms: {}".format(len(data)))
print("hit landfall: {}".format(landfall_count))

# create data frame
df = pd.DataFrame(data, columns=["name", "genesis_basin", "sub_basin", "time",
                                 "lon", "lat", "dist2land", "msw", "mcp", "nature",
                                 "track_type", "landfall"])

In [ ]:
display(df)

## Visualize the filtered storms

In [ ]:
# set styles
style = {
    "color": "red",
    "weight": 1,
}
hover_style = {
    "weight": 5,
}

# hover handler
def hover_handler(event=None, id=None, properties=None):
    sys.stdout.write("\r" + properties['msg'])
    sys.stdout.flush()

# show map    
m = Map(center=[0, 0], zoom=2)
m

In [ ]:
# add tracks
for i in data:
    l = GeoJSON(data=i['feature'], style=style, hover_style=hover_style)
    l.on_hover(hover_handler)
    m.add_layer(l)